In [ ]:
import json
import os
import matplotlib.pyplot as plt

In [ ]:
bench_dir = "../bench/k6"

for d in os.listdir(bench_dir):
    log_file = os.path.join(bench_dir, d, "console.log")
    config_file = os.path.join(bench_dir, d, "config.json")
    with open(config_file, "r") as f:
        config = json.loads(f.read())

    if "receipt" in config["func"]:
        continue
    if config["enclave"] != "sgx":
        continue
    if config["http_version"] != 2:
        continue
    if len(config["nodes"]) != 1:
        continue
    if config["content_type"] != "grpc":
        continue

    print(config)

    print(log_file)
    with open(log_file, "r") as f:

        times = []
        cterms = []
        crevs = []
        terms = []
        revs = []

        for line in f.readlines():
            line = line.strip()
            line = json.loads(line)
            time = line["time"]
            method = line["method"]
            if "header" not in line["res"]:
                continue
            cterm = int(line["res"]["header"]["committedRaftTerm"])
            crev = int(line["res"]["header"]["committedRevision"])
            term = int(line["res"]["header"]["raftTerm"])
            rev = int(line["res"]["header"]["revision"])
            # print(method, cterm, crev, term, rev)
            times.append(time)
            cterms.append(cterm)
            crevs.append(crev)
            terms.append(term)
            revs.append(rev)

        start = min(times)
        times = [t - start for t in times]

In [ ]:
fig1 = plt.figure(figsize=(6, 2))

times_s = [t / 1000 for t in times]

ax1 = fig1.add_subplot(111)
ax1.plot(times_s, revs, label="Latest")
ax1.plot(times_s, crevs, label="Committed")
ax1.set_xlabel("Time (s)")
ax1.set_ylabel("Revision")
ax1.legend()
plt.tight_layout()

x = []
y1 = []
y2 = []

for (t, r, c) in zip(times_s, revs, crevs):
    if 30 <= t <= 33:
        x.append(t)
        y1.append(r)
        y2.append(c)

ax2 = plt.axes([0.75, 0.45, 0.2, 0.25])
ax2.plot(x, y1)
ax2.plot(x, y2)

plt.savefig("../plots/k6/final-commit-latency.pdf")